In [1]:
#importing libraries
import pandas as pd

In [52]:
#reading file, merging date and time columns for actual and planned arrivals and departures
df = pd.read_csv('https://raw.githubusercontent.com/datacamp/careerhub-data/master/Belgium%20Train%20Times/belgium-train-times.csv', sep=';',
                parse_dates=[[16,5],[17,6],[14,7],[15,8]])

df.columns.values[:4] = ['Actual arrival datetime', 'Actual departure datetime',
                         'Planned arrival datetime', 'Planned departure datetime'] #changing the name of the merged columns

df.iloc[:,:4] = df.iloc[:,:4].apply(pd.to_datetime, errors='coerce') #conversion to datetime

df.head()

,Actual arrival datetime,Actual departure datetime,Planned arrival datetime,Planned departure datetime,Date of departure,Train number,Relation,Railway operator,Departure line,Delay at arrival,Delay at departure,Relation direction,Stopping place,Arrival line
0,2021-02-22 08:13:25,2021-02-22 08:13:25,2021-02-22 08:10:00,2021-02-22 08:10:00,2021-02-22,2528,IC 17,SNCB/NMBS,161,205.0,205.0,IC 17: DINANT -> BRUSSELS AIRPORT - ZAVENTEM,GROENENDAAL,161
1,2021-02-22 08:31:12,2021-02-22 08:31:12,2021-02-22 08:30:00,2021-02-22 08:30:00,2021-02-22,2528,IC 17,SNCB/NMBS,26,72.0,72.0,IC 17: DINANT -> BRUSSELS AIRPORT - ZAVENTEM,EVERE,26
2,2021-02-22 08:38:34,2021-02-22 08:38:34,2021-02-22 08:38:00,2021-02-22 08:38:00,2021-02-22,2528,IC 17,SNCB/NMBS,36,34.0,34.0,IC 17: DINANT -> BRUSSELS AIRPORT - ZAVENTEM,DIEGEM,36
3,2021-02-22 08:39:35,2021-02-22 08:39:35,2021-02-22 08:39:00,2021-02-22 08:39:00,2021-02-22,2528,IC 17,SNCB/NMBS,36,35.0,35.0,IC 17: DINANT -> BRUSSELS AIRPORT - ZAVENTEM,ZAVENTEM,36
4,2021-02-22 16:41:36,2021-02-22 16:42:06,2021-02-22 16:42:00,2021-02-22 16:42:00,2021-02-22,1487,L A4-1,SNCB/NMBS,59,-24.0,6.0,L A4-1: SINT-NIKLAAS -> ANTWERPEN-CENTRAAL,BEVEREN(WAAS),59
